In [ ]:
# Colab 전용 설치/환경 설정
# 1) 필수 패키지 설치 후 런타임 재시작 필요할 수 있습니다.
# 2) GOOGLE_API_KEY는 개인 키로 설정하세요.

# !pip -q install --upgrade langchain langchain-community langchain-google-genai faiss-cpu langgraph

import os
if "GOOGLE_API_KEY" not in os.environ or not os.environ["GOOGLE_API_KEY"]:
    # 사용 전 아래 주석을 해제하고 본인 키로 설정하세요. 설정 후 런타임 재시작이 필요할 수 있습니다.
    # os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"
    pass

print("GOOGLE_API_KEY set:", bool(os.environ.get("GOOGLE_API_KEY")))


In [ ]:
# Colab 환경 준비 및 Gemini 사용으로 전환
# 이 노트북은 Colab에서 실행되도록 설계되었습니다.
# 주의: OpenRouter/Neo4j 의존성을 제거하고, Google Gemini + FAISS를 사용합니다.

import os
import json
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END
from langchain_core.pydantic_v1 import Field

# 필수 라이브러리 설치 셀은 아래에 별도 제공합니다.
# GOOGLE_API_KEY는 환경변수로 설정해야 합니다.

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY", "")
if not GOOGLE_API_KEY:
    raise RuntimeError("GOOGLE_API_KEY 환경변수가 필요합니다. Colab에서 런타임 재시작 후 다시 실행하세요.")

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Gemini 모델 설정
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY)

# 간단한 로컬 코퍼스(예시) + FAISS로 대체
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

embeddings = GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY)

# 예시 전략 문서 (실제 프로젝트에서는 Colab 드라이브/파일에서 적재 가능)
STRATEGY_DOCS = [
    "볼린저 밴드 전략은 이동평균과 표준편차 밴드를 활용해 과매수/과매도 구간을 판단합니다.",
    "RSI 전략은 상대강도지수를 사용하여 과매수/과매도를 판별합니다.",
    "MACD 전략은 단기/장기 이동평균선의 차이와 시그널선을 비교합니다."
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = []
for doc in STRATEGY_DOCS:
    chunks.extend(text_splitter.split_text(doc))

vector_store = FAISS.from_texts(chunks, embedding=embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [ ]:
# ====================
# LangGraph 상태 및 간단 유틸 정의
# ====================
class AgentState(TypedDict):
    """LangGraph 워크플로우의 상태를 정의합니다."""
    query: str  # 사용자의 질문
    trading_data: Optional[str] = Field(None, description="사용자 매매 기록 데이터")
    strategy_info: Optional[str] = Field(None, description="검색된 투자 전략 정보")
    final_answer: Optional[str] = Field(None, description="최종 답변")
    error: Optional[str] = Field(None, description="에러 메시지")

# 간단 매매기록 조회 (더미)
def get_user_trading_data(user_input: str) -> str:
    dummy_trading_data = {
        "종목": "AAPL",
        "매수_2024-03-01": "170 USD, 10주",
        "매도_2024-05-15": "195 USD, 10주",
        "수익률": "+14.7%"
    }
    return json.dumps(dummy_trading_data, indent=2)

# FAISS 기반 간단 전략 검색
def search_strategy_info(query: str) -> str:
    docs = retriever.get_relevant_documents(query)
    return "\n\n".join([doc.page_content for doc in docs])

In [ ]:
# ====================
# LangGraph 노드 정의 (단순화)
# ====================

def retrieve_data_node(state: AgentState):
    trading_data = state.get("trading_data") or get_user_trading_data(state["query"])
    strategy_info = state.get("strategy_info") or search_strategy_info(state["query"])
    return {"trading_data": trading_data, "strategy_info": strategy_info}


def generate_response_node(state: AgentState):
    prompt = (
        f"당신은 개인 투자자의 매매 기록과 투자 전략을 분석하는 AI 전문가입니다. "
        f"아래 정보를 바탕으로 사용자의 질문에 대해 전문적인 조언을 제공하세요.\n\n"
        f"사용자 매매 기록: {state.get('trading_data')}\n\n"
        f"투자 전략 정보: {state.get('strategy_info')}\n\n"
        f"질문: {state['query']}\n\n"
        f"투자 조언:"
    )
    response = llm.invoke(prompt)
    return {"final_answer": response.content}


def router_node(state: AgentState) -> str:
    if state.get("trading_data") and state.get("strategy_info"):
        return "generate_response"
    return "retrieve_data"

In [ ]:
# ====================
# LangGraph 구축 및 실행
# ====================
workflow = StateGraph(AgentState)
workflow.add_node("retrieve_data", retrieve_data_node)
workflow.add_node("generate_response", generate_response_node)

workflow.add_conditional_edges(
    "retrieve_data",
    router_node,
    {
        "retrieve_data": "retrieve_data",
        "generate_response": "generate_response",
    },
)

workflow.set_entry_point("retrieve_data")
workflow.add_edge("generate_response", END)

app = workflow.compile()

# 실행 예시
initial_state = {"query": "내 AAPL 매매 기록을 바탕으로 볼린저 밴드 전략에 대해 조언해줘"}
result = app.invoke(initial_state)

print("\n\n--- 최종 결과 ---")
print(result.get("final_answer"))